In [2]:
import pandas as pd

In [3]:
raw_df=pd.read_csv('insurance.csv')

In [4]:
raw_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
inputs=raw_df[['age','sex','bmi','children','smoker','region']]
target=raw_df['charges']

In [6]:
numerical_cols=['age','bmi','children']

In [7]:
from sklearn.model_selection import train_test_split
inputs_train,input_test,target_train,target_test=train_test_split(inputs, target, test_size=0.25, random_state=42)

In [8]:
dict1={'no':0,'yes':1}

In [9]:
dict2={'male':1,'female':0}

In [10]:
dict3={'southeast':1,'southwest':2,'northeast':3,'northwest':4}

In [11]:
inputs_train['smoker']=inputs_train.smoker.map(dict1)
input_test['smoker']=input_test.smoker.map(dict1)

In [12]:
inputs_train['sex']=inputs_train.sex.map(dict2)
input_test['sex']=input_test.sex.map(dict2)

In [13]:
inputs_train['region']=inputs_train.region.map(dict3)
input_test['region']=input_test.region.map(dict3)

In [14]:
input_test

,age,sex,bmi,children,smoker,region
764,45,0,25.175,2,0,3
887,36,0,30.020,0,0,4
890,64,0,26.885,0,1,4
1293,46,1,25.745,3,0,4
259,19,1,31.920,0,1,4
...,...,...,...,...,...,...
342,60,0,27.550,0,0,3
308,58,1,34.865,0,0,3
1128,34,1,32.800,1,0,2
503,19,1,30.250,0,1,1


In [15]:
inputs_train

,age,sex,bmi,children,smoker,region
693,24,1,23.655,0,0,4
1297,28,0,26.510,2,0,1
634,51,1,39.700,1,0,2
1022,47,1,36.080,1,1,1
178,46,0,28.900,2,0,2
...,...,...,...,...,...,...
1095,18,0,31.350,4,0,3
1130,39,0,23.870,5,0,1
1294,58,1,25.175,0,0,3
860,37,0,47.600,2,1,2


In [16]:
from xgboost import XGBRegressor

In [17]:
model=XGBRegressor(random_state=42)

In [18]:
model.fit(inputs_train,target_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [19]:
preds=model.predict(inputs_train)

In [20]:
preds

array([ 2404.627 ,  3863.4333,  8968.704 , ..., 11674.673 , 45935.145 ,
       10893.275 ], dtype=float32)

In [21]:
from sklearn.metrics import r2_score

In [22]:
lossy=r2_score(target_train,preds)

In [23]:
lossy

0.9954040703345491

In [24]:
predict=model.predict(input_test)

In [25]:
lossy2=r2_score(target_test,predict)

In [26]:
lossy2

0.8348220378696097

In [27]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'n_estimators': [100, 200, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'max_depth': [3, 5, 7, 10],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'reg_alpha': [0, 0.1, 1, 10],
    'reg_lambda': [0.01, 0.1, 1],
    'gamma': [0, 0.1, 0.5, 1],
}
print(param_grid)

{'n_estimators': [100, 200, 500, 1000], 'learning_rate': [0.01, 0.05, 0.1, 0.3], 'max_depth': [3, 5, 7, 10], 'min_child_weight': [1, 3, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.5, 0.7, 1.0], 'reg_alpha': [0, 0.1, 1, 10], 'reg_lambda': [0.01, 0.1, 1], 'gamma': [0, 0.1, 0.5, 1]}


In [29]:
xg=XGBRegressor()
xg_randomcv=RandomizedSearchCV(estimator=xg,param_distributions=param_grid,n_iter=100,cv=3,verbose=2,
                               random_state=100,n_jobs=-1)
xg_randomcv.fit(inputs_train,target_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.5, 0.7, 1.0],
                                        'gamma': [0, 0.1, 0.5, 1],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.3],
                                        'max_depth': [3, 5, 7, 10],
                                        'min_child_weight': [1, 3, 5],
                                        'n_estimators': [100, 200, 500, 1000],
                                        'reg_alpha': [0, 0.1, 1, 10],
                                        'reg_lambda': [0.01, 0.1, 1],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=100, verbose=2)

In [30]:
xg_randomcv.best_params_

{'subsample': 0.8,
 'reg_lambda': 1,
 'reg_alpha': 10,
 'n_estimators': 500,
 'min_child_weight': 1,
 'max_depth': 3,
 'learning_rate': 0.01,
 'gamma': 1,
 'colsample_bytree': 1.0}

In [31]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [450, 500, 550],
    'learning_rate': [0.005, 0.008, 0.01,0.0015],
    'max_depth': [2, 3, 6],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.9, 1, 1.5],
    'gamma': [1, 2, 3],
    'reg_alpha': [10, 15,20],
    'reg_lambda': [1,2, 3]
}

In [32]:
grid_search = GridSearchCV(estimator=XGBRegressor(objective='reg:squarederror', random_state=42),
                           param_grid=param_grid, 
                           scoring='neg_mean_squared_error', 
                           cv=5, 
                           verbose=2, 
                           n_jobs=-1)

grid_search.fit(inputs_train,target_train)

Fitting 5 folds for each of 8748 candidates, totalling 43740 fits


C:\Users\VIVEK KUMAR\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
14580 fits failed out of a total of 43740.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
14580 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\VIVEK KUMAR\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\VIVEK KUMAR\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
  File "C:\Users\VIVEK KUMAR\AppData\Local\Programs\Python\Python310\lib\site

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.9, 1, 1.5], 'gamma': [1, 2, 3],
                         'learning_rate': [0.005, 0.008, 0.01, 0.0015],
                         'max_depth': [2, 3, 6],
                         'n_estimators': [450, 500, 550],
                         'reg_alpha': [10, 15, 20], 'reg_lambda': [1, 2, 3],
                         'subsample': [0.7, 0.8, 0.9]},
             scoring='neg_mean_squared_error', verbose=2)

In [33]:
grid_search.best_params_

{'colsample_bytree': 0.9,
 'gamma': 1,
 'learning_rate': 0.01,
 'max_depth': 2,
 'n_estimators': 550,
 'reg_alpha': 20,
 'reg_lambda': 1,
 'subsample': 0.7}

In [36]:
xg_best=grid_search.best_estimator_

In [37]:
xg_best.fit(inputs_train,target_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=550, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [38]:
preds=xg_best.predict(inputs_train)

In [39]:
preds

array([ 4751.167,  5881.738, 11464.561, ..., 12435.1  , 40532.242,
       12143.261], dtype=float32)

In [40]:
lossy=r2_score(target_train,preds)

In [41]:
lossy

0.8716131559380954

In [42]:
predict=xg_best.predict(input_test)

In [43]:
predict

array([10457.517 ,  6279.548 , 26921.408 , 10449.298 , 35671.066 ,
        6832.965 ,  2795.2817, 14712.339 ,  4634.2065, 11654.432 ,
       19637.516 ,  7984.0376,  5340.2485, 43505.64  , 45512.145 ,
       43529.52  , 11815.161 , 43392.305 ,  9730.881 , 23013.732 ,
        6507.1216,  9905.869 ,  2687.8042,  4230.6816, 11882.615 ,
       12599.917 , 14696.954 ,  6583.483 , 11338.555 ,  2747.6343,
        7988.7803, 12680.135 ,  3649.7158,  5845.093 ,  5405.4824,
        9908.387 ,  3938.5286,  9248.481 , 25029.941 , 39367.223 ,
        4933.004 ,  4663.6226, 13463.48  , 13993.337 ,  7384.7393,
       15549.143 ,  5684.281 ,  5854.216 , 41731.07  ,  6687.448 ,
       14788.354 ,  3445.5566,  8136.8257,  2908.6255, 12575.031 ,
       12123.507 ,  4904.623 , 38968.11  , 12553.505 , 12222.655 ,
       14510.4795,  6786.6006, 16866.467 ,  8145.8716, 11336.869 ,
        5789.02  , 19296.688 , 11838.49  ,  5445.2363,  3634.939 ,
        8005.1025, 10770.251 , 10616.156 ,  7618.866 ,  9918.3

In [44]:
lossy2=r2_score(target_test,predict)

In [45]:
lossy2

0.8690339440158149

In [46]:
import numpy as np
def rmse(targets, predictions):
    return np.sqrt(np.mean(np.square(targets - predictions)))

In [48]:
train_rmse=rmse(target_train,preds)
test_rmse=rmse(target_test, predict)

In [49]:
train_rmse

4315.9816688817245

In [50]:
test_rmse

4445.404081752918